# Recessions

#### Load python tools

In [1]:
%load_ext lab_black

In [5]:
import altair as alt
import altair_stiles as altstiles
import pandas as pd
import requests
import json

In [6]:
pd.options.display.max_columns = 1000
pd.options.display.max_rows = 1000
alt.data_transformers.disable_max_rows()
alt.themes.register("stiles", altstiles.theme)
alt.themes.enable("stiles")

ThemeRegistry.enable('stiles')

----

## Get data

#### Recessions from Wikipedia

In [7]:
headers = {
    "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36"
}

In [8]:
url = 'https://fred.stlouisfed.org/graph/api/series/?obs=true&sid=T10Y2Y'

In [9]:
requests.get(url, headers=headers)

<Response [422]>

In [ ]:
pd.read_json('https://fred.stlouisfed.org/graph/api/series/?obs=true&sid=T10Y2Y')

In [ ]:
src = pd.read_html(
    "https://en.wikipedia.org/wiki/List_of_recessions_in_the_United_States"
)[2]

#### Clean up columns

In [5]:
src.columns = (
    src.columns.str.lower()
    .str.replace(" ", "_")
    .str.replace("_(peak_to_trough)", "", regex=False)
)

#### Slim the dataframe to columns we need

In [6]:
df = src[["name", "period_range", "gdp_decline"]].copy()

#### Split the period range to start and end columns

In [7]:
df[["start", "end"]] = (
    df["period_range"].str.split("[").str[0].str.split("–", expand=True)
)

In [8]:
df.drop(["period_range"], axis=1, inplace=True)

#### Clean up GDP figure

In [9]:
df["gdp_decline"] = (
    df["gdp_decline"]
    .str.split("[")
    .str[0]
    .str.replace("%", "", regex=False)
    .str.replace("−", "", regex=False)
    .astype(float)
) * -1

#### Dates

In [10]:
df["start"] = pd.to_datetime(df["start"])
df["end"] = pd.to_datetime(df["end"])

/var/folders/k0/l0l0k9f50w7cg_phqmhkhybm0000gx/T/ipykernel_90712/2446664777.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["start"] = pd.to_datetime(df["start"])
/var/folders/k0/l0l0k9f50w7cg_phqmhkhybm0000gx/T/ipykernel_90712/2446664777.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["end"] = pd.to_datetime(df["end"])


In [11]:
df["start_year"] = df["start"].dt.year.astype(str).str.replace(".0", "", regex=False)

In [12]:
df["type"] = "Recessions"

In [13]:
df["gdp_decline_pos"] = df["gdp_decline"] * -1

In [14]:
df = df.drop(0)

---

## Charts

#### Recessions

In [15]:
df

,name,gdp_decline,start,end,start_year,type,gdp_decline_pos
1,Great Depression,-26.7,1929-08-01,1933-03-01,1929,Recessions,26.7
2,Recession of 1937–1938,-18.2,1937-05-01,1938-06-01,1937,Recessions,18.2
3,Recession of 1945,-12.7,1945-02-01,1945-10-01,1945,Recessions,12.7
4,Recession of 1949,-1.7,1948-11-01,1949-10-01,1948,Recessions,1.7
5,Recession of 1953,-2.6,1953-07-01,1954-05-01,1953,Recessions,2.6
6,Recession of 1958,-3.7,1957-08-01,1958-04-01,1957,Recessions,3.7
7,Recession of 1960–1961,-1.6,1960-04-01,1961-02-01,1960,Recessions,1.6
8,Recession of 1969–1970,-0.6,1969-12-01,1970-11-01,1969,Recessions,0.6
9,1973–1975 recession,-3.2,1973-11-01,1975-03-01,1973,Recessions,3.2
10,1980 recession,-2.2,1980-01-01,1980-07-01,1980,Recessions,2.2


In [16]:
rect = (
    alt.Chart(df, padding={"left": 10})
    .mark_rect(color="#e6e6e6")
    .encode(x="start:T", x2="end:T")
)

rect.properties(title="US recessions since 1929")

alt.Chart(...)

#### Recessions

In [17]:
df

,name,gdp_decline,start,end,start_year,type,gdp_decline_pos
1,Great Depression,-26.7,1929-08-01,1933-03-01,1929,Recessions,26.7
2,Recession of 1937–1938,-18.2,1937-05-01,1938-06-01,1937,Recessions,18.2
3,Recession of 1945,-12.7,1945-02-01,1945-10-01,1945,Recessions,12.7
4,Recession of 1949,-1.7,1948-11-01,1949-10-01,1948,Recessions,1.7
5,Recession of 1953,-2.6,1953-07-01,1954-05-01,1953,Recessions,2.6
6,Recession of 1958,-3.7,1957-08-01,1958-04-01,1957,Recessions,3.7
7,Recession of 1960–1961,-1.6,1960-04-01,1961-02-01,1960,Recessions,1.6
8,Recession of 1969–1970,-0.6,1969-12-01,1970-11-01,1969,Recessions,0.6
9,1973–1975 recession,-3.2,1973-11-01,1975-03-01,1973,Recessions,3.2
10,1980 recession,-2.2,1980-01-01,1980-07-01,1980,Recessions,2.2


In [18]:
bubbles = (
    alt.Chart(df)
    .mark_circle(color="#7c4ea5")
    .encode(
        alt.X("start_year:O", axis=alt.Axis(), title=" "),
        alt.Y("type:N", title=" "),
        alt.Size(
            "gdp_decline_pos:Q",
            scale=alt.Scale(range=[0, 3000]),
            legend=None,
        ),
    )
    .properties(width=650, height=120)
)

text = (
    alt.Chart(df)
    .mark_text(dy=-40, color="black")
    .encode(
        x=alt.X("start_year:O"),
        y=alt.Y("type:N"),
        text=alt.Text("gdp_decline:Q", format=""),
    )
)

text2 = (
    alt.Chart(df.query("name=='Great Depression' | name=='COVID-19 recession'"))
    .mark_text(dy=40, color="black")
    .encode(
        x=alt.X("start_year:O"),
        y=alt.Y("type:N"),
        text=alt.Text("name", title=" "),
    )
)

(bubbles + text + text2).properties(
    title="U.S. recessions by start year and GDP decline"
)

alt.LayerChart(...)

---

In [19]:
df.to_csv("data/processed/recessions_1929_current.csv", index=False)